# **This notebook is created to use the arithmatic robo environment and train it using random numbers.**

The environment generates two numbers randomly. What we want our agent is to find the most efficient operation to go from number A to number B.

Agent learns to choose right operation among division, multiplication, addition and subtraction by randomly selecting these operations randomly.

If the chosen action is right then reward is given else small penalty is deducted.

This logic is built in arithmaticrobo_env.py 

We use this notebook to create an instance of environment, generate actions randomly and train the agent. Two types of agent have been tried.

Use when you get code from git repo. For the first time copy the git repo.

In [1]:
!git clone https://github.com/gahanh/arithmatic-robo.git 

Cloning into 'arithmatic-robo'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 29 (delta 4), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (29/29), done.


This is to be used when you copy the code from Google Drive. This will be when you want to change the code yourself.

In that case get the zip file form github repo , unzip locally and then upload the whole folder on to your Google Drive. 

Then change the environment as you like and execute the following block.

In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)
!cp  '/gdrive/My Drive/arithmatic-robo-master-1/' /content/

Mounted at /gdrive


In [2]:
!pip install keras-rl

     |████████████████████████████████| 40kB 2.9MB/s 
  Created wheel for keras-rl: filename=keras_rl-0.4.2-cp37-none-any.whl size=48382 sha256=b0e433b4d61ea3f4bd59834f6cdf7aefa55a0598b4e2cf8edc6f9ab53280661b
  Stored in directory: /root/.cache/pip/wheels/7d/4d/84/9254c9f2e8f51865cb0dac8e79da85330c735551d31f73c894
Successfully built keras-rl


In [3]:
cd /content/arithmatic-robo/ArithmaticRobo-Gym_Environment/gym-arithmaticrobo


/content/arithmatic-robo/ArithmaticRobo-Gym_Environment/gym-arithmaticrobo


In [4]:
!pip install -e .

Obtaining file:///content/arithmatic-robo/ArithmaticRobo-Gym_Environment/gym-arithmaticrobo
  Running setup.py develop for gym-tictac4


In [2]:
import gym
import gym_arithmaticrobo
from gym import spaces

env = gym.make('arithmaticrobo-v0')

In init


In [3]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam

from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

from rl.agents.cem import CEMAgent
from rl.memory import EpisodeParameterMemory

import numpy as np


In [4]:
model = Sequential()
model.add(Flatten(input_shape=(1,2)))
model.add(Dense(4))
model.add(Activation('softmax'))

In [5]:
from rl.agents import SARSAAgent
policy = BoltzmannQPolicy()
sarsa = SARSAAgent(model=model, nb_actions=4, nb_steps_warmup=500, policy=policy)
sarsa.compile(Adam(lr=1e-3), metrics=['mae'])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [6]:
sarsa.fit(env, nb_steps=3000, visualize=False, verbose=1)

Streaming output truncated to the last 5000 lines.
150 244
In ADD Reward
150 244
In SUBTRACT
151 244
subtract none
 1765/10000 [====>.........................] - ETA: 4:01 - reward: 1.6918In Divide
151 244
0.6188524590163934
div none
In SUBTRACT
151 244
subtract none
 1767/10000 [====>.........................] - ETA: 4:01 - reward: 1.6887In SUBTRACT
151 244
subtract none
In ADD
151 244
In ADD Reward
151 244
 1769/10000 [====>.........................] - ETA: 4:01 - reward: 1.6874In Divide
152 244
0.6229508196721312
div none
In Divide
152 244
0.6229508196721312
div none
 1771/10000 [====>.........................] - ETA: 4:01 - reward: 1.6844In Divide
152 244
0.6229508196721312
div none
In Divide
152 244
0.6229508196721312
div none
 1773/10000 [====>.........................] - ETA: 4:00 - reward: 1.6813In Divide
152 244
0.6229508196721312
div none
In Multiply
152 244
1.605263157894737
In MULTIPLY Reward
152 244
 1775/10000 [====>.........................] - ETA: 4:00 - reward: 1.6817I

In [19]:
history = sarsa.test(env, nb_episodes=1, visualize=False)

Testing for 1 episodes ...
In Reset
In Multiply
139 229
1.6474820143884892
In MULTIPLY Reward
139 229
In Divide
229 229
Game Over
Episode 1: reward: 10.000, steps: 2


In [ ]:
history

In [ ]:
rewards = np.array(history.history['episode_reward'])
print(("Test rewards (#episodes={}): mean={:>5.2f}, std={:>5.2f}, "
           "min={:>5.2f}, max={:>5.2f}")
          .format(len(rewards),
                  rewards.mean(),
                  rewards.std(),
                  rewards.min(),
                  rewards.max()))

Test rewards (#episodes=300): mean=51.53, std=69.52, min= 0.00, max=301.00


In [ ]:
memory = EpisodeParameterMemory(limit=30000, window_length=1)

cem = CEMAgent(model=model, nb_actions=4, memory=memory,
               batch_size=10, nb_steps_warmup=500, train_interval=50, elite_frac=0.05)
cem.compile()

In [ ]:
cem.fit(env, nb_steps=10000, visualize=False, verbose=2)

Streaming output truncated to the last 5000 lines.
86 87
subtract none
In SUBTRACT
86 87
subtract none
In Multiply
86 87
1.0116279069767442
In MULTIPLY Reward
86 87
In SUBTRACT
87 87
Game Over
 8791/10000: episode: 1501, duration: 0.034s, episode steps: 6, steps per second: 174, episode reward: 12.000, mean reward: 2.000 [-1.000, 5.000], mean action: 1.500 [1.000, 3.000], mean observation: 86.667 [86.000, 87.000], mean_best_reward: --
In Reset
In ADD
120 88
add none
In Multiply
120 88
0.7333333333333333
mul none
In Divide
120 88
1.3636363636363635
In DIVIDE Reward
120 88
In SUBTRACT
88 88
Game Over
 8795/10000: episode: 1502, duration: 0.031s, episode steps: 4, steps per second: 131, episode reward: 8.000, mean reward: 2.000 [-1.000, 5.000], mean action: 1.500 [0.000, 3.000], mean observation: 96.000 [88.000, 120.000], mean_best_reward: --
In Reset
In Multiply
118 117
0.9915254237288136
mul none
In ADD
118 117
add none
In SUBTRACT
118 117
In SUBTRACT Reward
118 117
In ADD
117 117
Game 

In [ ]:
cem.test(env, nb_episodes=1, visualize=False)

Streaming output truncated to the last 5000 lines.
In Multiply
99 80
0.8080808080808081
mul none
In Multiply
99 80
0.8080808080808081
mul none
In Multiply
99 80
0.8080808080808081
mul none
In Multiply
99 80
0.8080808080808081
mul none
In Multiply
99 80
0.8080808080808081
mul none
In Multiply
99 80
0.8080808080808081
mul none
In Multiply
99 80
0.8080808080808081
mul none
In Multiply
99 80
0.8080808080808081
mul none
In Multiply
99 80
0.8080808080808081
mul none
In Multiply
99 80
0.8080808080808081
mul none
In Multiply
99 80
0.8080808080808081
mul none
In Multiply
99 80
0.8080808080808081
mul none
In Multiply
99 80
0.8080808080808081
mul none
In Multiply
99 80
0.8080808080808081
mul none
In Multiply
99 80
0.8080808080808081
mul none
In Multiply
99 80
0.8080808080808081
mul none
In Multiply
99 80
0.8080808080808081
mul none
In Multiply
99 80
0.8080808080808081
mul none
In Multiply
99 80
0.8080808080808081
mul none
In Multiply
99 80
0.8080808080808081
mul none
In Multiply
99 80
0.808080808

KeyboardInterrupt: ignored

In [ ]:
#print(env.done, env.x, env.y)
#print(env.action_space)
env.reset()
for i in range(5):
    action = env.action_space.sample()
    env.step(action)
    if(env.done==1):
            break
    

In Reset
In SUBTRACT
189 103
In SUBTRACT Reward
189 103
In SUBTRACT
188 103
In SUBTRACT Reward
188 103
In SUBTRACT
187 103
In SUBTRACT Reward
187 103
In Divide
186 103
1.8058252427184467
In DIVIDE Reward
186 103
In ADD
102 103
In ADD Reward
102 103


In [ ]:
for i_episode in range(20):
    observation = env.reset()
    while(env.done==0):
        #env.render()
        print(observation)
        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)
env.close()

In Reset
[0. 0.]
In ADD
174 241
In ADD Reward
174 241
[175. 241.]
In SUBTRACT
175 241
subtract none
[175. 241.]
In SUBTRACT
175 241
subtract none
[175. 241.]
In Multiply
175 241
1.3771428571428572
In MULTIPLY Reward
175 241
[241. 241.]
In Multiply
241 241
Game Over
In Reset
[0. 0.]
In ADD
142 101
add none
[142. 101.]
In Divide
142 101
1.4059405940594059
In DIVIDE Reward
142 101
[101. 101.]
In Multiply
101 101
Game Over
In Reset
[0. 0.]
In Multiply
159 169
1.0628930817610063
In MULTIPLY Reward
159 169
[169. 169.]
In ADD
169 169
Game Over
In Reset
[0. 0.]
In SUBTRACT
98 200
subtract none
[ 98. 200.]
In SUBTRACT
98 200
subtract none
[ 98. 200.]
In Multiply
98 200
2.0408163265306123
In MULTIPLY Reward
98 200
[200. 200.]
In Multiply
200 200
Game Over
In Reset
[0. 0.]
In Divide
176 222
0.7927927927927928
div none
[176. 222.]
In Divide
176 222
0.7927927927927928
div none
[176. 222.]
In ADD
176 222
In ADD Reward
176 222
[177. 222.]
In Multiply
177 222
1.2542372881355932
In MULTIPLY Reward
177 

In [ ]:
int(round(1.6))

2